In [1]:
import duckdb

In [8]:
%pwd

'/Users/charlescurtin/bren-meds213-spring-2024/week3'

1. Create a connection and a cursor

In [2]:
conn = duckdb.connect("database.db")

In [3]:
conn

In [4]:
cur = conn.cursor()

In [16]:
cur

Now let's do something with our cursor

In [17]:
cur.execute("SELECT * FROM Site LIMIT 5")

In [18]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

In [19]:
# things are returned in a tuple, even if they are single values
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

# select the single value from each tuple
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

2. Get the one result, or the next result

In [20]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall()

[(1547,)]

In [21]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

(1547,)

In [22]:
# extract the element from the tuple
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0]

1547

3. Using an iterator- but duckdb doesnt support iterators

In [23]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
        # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


another version of things- one string that spans multiple lines

In [24]:
# triple double quote- in python a string can't span multiple lines, but it lets you do it with the triple double quote
cur.execute("""
    CREATE TEMP TABLE temp_table AS
    SELECT * FROM Bird_nests LIMIT 10
""")

In [26]:
cur.execute("SELECT * FROM temp_table")

In [27]:
cur.fetchall()

[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

- A note on fragility

For example:

INSERT INTO Site VALUES ("abcd", "foo", 35.7, -119.5)
- the order of these values must be in the same order as the columns they're being inserted into

- A less fragile and more explicit way:

INSERT INTO Site (Code, Site_name, Latitude, Longtitude)
     VALUES ("abcd", "foo", 35.7, -119.5)
     
- in the same vein: SELECT * is fragile
- a  better more robust way of coding the same thing:

In [29]:
cur.execute("SELECT Site_name, Code, Latitude, Longitude, FROM Site LIMIT 3")
cur.fetchall()

[('Barrow', 'barr', 71.30000305175781, -156.60000610351562),
 ('Burntpoint Creek', 'burn', 55.20000076293945, -84.30000305175781),
 ('Bylot Island', 'bylo', 73.19999694824219, -80.0)]

## how many nests do we have for each species?

Approach: get all species. Then execute a count query for each species

In [33]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
"""
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Another way to do it:

In [34]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = ?
"""
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    #prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code])
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Let's illustrate the danger of the first version of doing it

In [35]:
abbrev = "TS"
name = "Taylor Swift"
cur.execute("""
    INSERT INTO Personnel (Abbreviation, Name)
    VALUES ('%s', '%s')
    """ % (abbrev, name)
           )

In [36]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('emagnuson', 'Emily Magnuson'),
 ('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift')]

In [37]:
abbrev = "CO"
name = "Conan O'Brian"
cur.execute("""
    INSERT INTO Personnel (Abbreviation, Name)
    VALUES (?, ?)
    """,
            [abbrev, name])

In [38]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brian")]

### Wednesday, May 8

In [5]:
cur.execute("SELECT * FROM Camp_assignment LIMIT 3")
cur.fetchall()

[(2005,
  'bylo',
  'lmckinnon',
  datetime.date(2005, 6, 1),
  datetime.date(2005, 8, 5)),
 (2005,
  'bylo',
  'blalibert',
  datetime.date(2005, 6, 1),
  datetime.date(2005, 8, 20)),
 (2006,
  'bylo',
  'lmckinnon',
  datetime.date(2006, 6, 1),
  datetime.date(2006, 8, 5))]

In [6]:
cur.execute("SELECT * FROM Camp_assignment LIMIT 3")
cur.fetchone()

(2005,
 'bylo',
 'lmckinnon',
 datetime.date(2005, 6, 1),
 datetime.date(2005, 8, 5))

#### creating a parameterized query

In [9]:
inner_query = """
    SELECT COUNT (*) AS num_nests
    FROM Bird_nests
    WHERE Observer = ?
"""

# one query, we get the distinct observers from bird nests
outer_query = """
    SELECT DISTINCT Observer FROM Bird_nests
"""

# we iterate over the names of observers, then cur2 lets us conduct the inner query for each of those names 
for row in cur.execute(outer_query).fetchall():
    observer = row[0]
    cur2 = conn.cursor()
    cur2.execute(inner_query, [observer])
    print(f"Observer {observer} gathered {cur2.fetchone()[0]} nests")

Observer mballvanzee gathered 2 nests
Observer dkessler gathered 69 nests
Observer bharrington gathered 245 nests
Observer lmckinnon gathered 249 nests
Observer dhodkinson gathered 15 nests
Observer mbwunder gathered 4 nests
Observer None gathered 0 nests
Observer kkalasz gathered 12 nests
Observer bhill gathered 55 nests
Observer ssaalfeld gathered 13 nests
Observer wenglish gathered 18 nests
Observer lworing gathered 14 nests
Observer vloverti gathered 54 nests
Observer rlanctot gathered 40 nests
Observer abankert gathered 17 nests
Observer amould gathered 42 nests
Observer bkaselow gathered 4 nests
Observer jflamarre gathered 43 nests
Observer edastrous gathered 38 nests
Observer jzamuido gathered 11 nests


## Pandas

You can use pandas commands to import as a dataframe you can use directly.

In [10]:
import pandas as pd

In [11]:
df = pd.read_sql("SELECT * FROM Site", conn)

In [12]:
df

,Code,Site_name,Location,Latitude,Longitude,Area
0,barr,Barrow,"Alaska, USA",71.300003,-156.600006,220.399994
1,burn,Burntpoint Creek,"Ontario, Canada",55.200001,-84.300003,63.000000
2,bylo,Bylot Island,"Nunavut, Canada",73.199997,-80.000000,723.599976
3,cakr,Cape Krusenstern,"Alaska, USA",67.099998,-163.500000,54.099998
4,cari,Canning River Delta,"Alaska, USA",70.099998,-145.800003,722.000000
5,chau,Chaun River Delta,"Chukotka, Russia",68.800003,170.600006,248.199997
6,chur,Churchill,"Manitoba, Canada",58.700001,-93.800003,866.900024
7,coat,Coats Island,"Nunavut, Canada",62.900002,-82.500000,1239.099976
8,colv,Colville River Delta,"Alaska, USA",70.400002,-150.699997,324.799988
9,eaba,East Bay,"Nunavut, Canada",64.000000,-81.699997,1205.500000
